In [247]:
import json

with open("data/operators.json", encoding="utf-8") as f:
    operator_data = json.load(f)

In [248]:
def sanitize_plan(farm_plan):
    sanitized_farm_plan = []
    for item in farm_plan:
        if item.startswith('e'):
            plan = parse_elite(item)
        elif item.startswith('s'):
            plan = parse_mastery(item)
        elif item.startswith(tuple(range(8))):
            plan = {
                'category': 'skillLevel',
                'skillLevel': int(item)
            }
        else:
            plan = None
        
        if plan:
            sanitized_farm_plan.append(plan)
    return sanitized_farm_plan

            
def parse_elite(elite):
    try:
        elite_lvl = int(elite[1:])
        if elite_lvl not in range(1, 3):
            print(f"Item {item} has an invalid elite level. Please only specify elite 1 or 2. Skipping this item.")
            return None
        return {
            'category': 'elite',
            'eliteLevel': elite_lvl
        }
    except ValueError:
        print(f"Item {elite} has an invalid elite level. Please only specify elite 1 or 2. Skipping this item.")
        return None
            
def parse_mastery(mastery):
    try:
        skill = int(mastery[1])
        mastery_lvl = int(mastery[3])
        if skill not in range(1, 4) or mastery_lvl not in range(1, 4):
            print(f"Item {item} has an invalid skill/mastery level. Skipping this item.")
            return None
        return {
            'category': 'mastery',
            'skillIndex': skill - 1,
            'masteryLevel': mastery_lvl
        }
    except ValueError:
        print(f"Mastery {mastery} is improperly formatted. It should look something like s2m1.")
        return None

with open("to_farm.txt", 'r') as f:
    xs = [line.rstrip() for line in f.readlines()]
print(xs)
operator_farm_plan = {}
for operator in xs:
    op_and_farm_plans = operator.split(',')
    op_name, op_farm_plan = op_and_farm_plans[0], op_and_farm_plans[1:]
    operator_farm_plan[op_name] = sanitize_plan(op_farm_plan)

operator_farm_plan['Surtr']

['Exusiai,s3m2,s3m3', 'Surtr,s3m2,s3m3', 'Angelina,s3m2,s3m3', 'Ash,e2,s2m1,s2m2,s3m3', 'Eyjafjalla,s2m1,s3m1,s3m2,s3m3', 'Ifrit,s3m1,s3m2,s3m3', 'Saria,s3m1,s3m2,s3m3,s1m2m,s3m3', 'Saga,s2m1,s3m1,s3m2,s3m3', 'Nightingale,e2', 'Ptilopsis,e2', 'SilverAsh,e1,e2,s3m1,s3m2,s3m3', 'Specter,e1,e2,s2m1,s2m2,s2m3', 'Shamare,e1,e2,s2m1,s2m2,s2m3', 'Phantom,e2,s2m1,s2m2,s2m3', 'Blaze,e1,e2,s2m1,s2m2,s2m3', 'Skadi the Corrupting Heart,e1,e2,s2m1,s2m2,s2m3,s3m1,s3m2,s3m3', 'Bagpipe,e1,e2,s3m1,s3m2,s3m3', 'Amiya,e2', 'Myrtle,e2,s1m1,s1m2,s1m3']


[{'category': 'mastery', 'skillIndex': 2, 'masteryLevel': 2},
 {'category': 'mastery', 'skillIndex': 2, 'masteryLevel': 3}]

In [249]:
import pandas as pd

op_df = pd.read_json('data/operators.json')


In [250]:
#op_df.head(5)["skills"][0][0]['masteries']
op_df.head(5)

,id,name,isCnOnly,rarity,class,skillLevels,elite,skills
0,char_002_amiya,Amiya,False,5,Caster,"[{'skillLevel': 2, 'ingredients': [{'id': '330...","[{'eliteLevel': 1, 'ingredients': [{'id': '400...","[{'skillId': 'skcom_magic_rage[3]', 'iconId': ..."
1,char_003_kalts,Kal'tsit,True,6,Medic,"[{'skillLevel': 2, 'ingredients': [{'id': '330...","[{'eliteLevel': 1, 'ingredients': [{'id': '400...","[{'skillId': 'skchr_kalts_1', 'iconId': None, ..."
2,char_009_12fce,12F,False,2,Caster,[],[],[]
3,char_010_chen,Ch'en,False,6,Guard,"[{'skillLevel': 2, 'ingredients': [{'id': '330...","[{'eliteLevel': 1, 'ingredients': [{'id': '400...","[{'skillId': 'skchr_chen_1', 'iconId': None, '..."
4,char_017_huang,Blaze,False,6,Guard,"[{'skillLevel': 2, 'ingredients': [{'id': '330...","[{'eliteLevel': 1, 'ingredients': [{'id': '400...","[{'skillId': 'skchr_huang_1', 'iconId': 'skcom..."


In [251]:
from bidict import bidict

item_path = "data/item_table.json"

with open(item_path, encoding="utf-8") as f:
    item_json = json.load(f)

item_by_id = bidict()
    
for key, value in item_json['items'].items():
    try:
        item_by_id[key] = value['name']
    except:
        continue

operator_farm_plan['Exusiai']

[{'category': 'mastery', 'skillIndex': 2, 'masteryLevel': 2},
 {'category': 'mastery', 'skillIndex': 2, 'masteryLevel': 3}]

In [264]:
import json

from collections import Counter
from pathlib import Path
from typing import List, Tuple

import pandas as pd

from bidict import bidict


class OperatorMaterialAggregator:
    """Load all operators' material usage by parsing the json data."""
    def __init__(self, data_path="data"):
        operator_json_path = Path(data_path) / "operators.json"
        item_path = Path(data_path) / "item_table.json"
        with open(operator_json_path, encoding="utf-8") as f:
            self._operator_json = json.load(f)
        with open(item_path, encoding="utf-8") as f2:
            self._item_json = json.load(f2)
            self._item_by_id = bidict()
    
            for key, value in self._item_json['items'].items():
                try:
                    self._item_by_id[key] = value['name']
                except:
                    continue
    
        self._op_df = pd.read_json(operator_json_path)

    
    def aggregate_mats_by_rarity(self, rarities=[6]):
        mats = Counter()
        for rarity in rarities:
            df = self._op_df[self._op_df['rarity'] == rarity]
            self.aggregate_mats(mats, df)
        return mats
    
    def aggregate_mats(self, mats: Counter, df: pd.DataFrame):
        for idx, row in df.iterrows():
            self.agg_skills(mats, row["skillLevels"])
            # self.agg_masteries(mats, row["skills"])

    def agg_skills(self, mats: Counter, skill_levels: List):
        for sk_lvl in skill_levels:
            ingredients = {
                ingredient['id']: ingredient['quantity']
                for ingredient in sk_lvl['ingredients']
            }
            mats.update(ingredients)
           # for ingredient in ingredients:
               # mats[ingredient['id']] += ingredient['quantity']
    
    def agg_mats_for_farm_plan(self, op_farm_plan):
        overall_mats = Counter()
        overall_mats_by_op = {}

        for op_name, plan in op_farm_plan.items():
            overall_mats_by_op[op_name] = Counter()
            op_mats = overall_mats_by_op[op_name]
            # print(op_name)
            op_series = self._op_df[self._op_df['name'] == op_name].iloc[0]
            for subplan in plan:
                self.agg_mats_for_subplan(op_series, op_mats, subplan)
            overall_mats.update(op_mats)
        return overall_mats, overall_mats_by_op
            
    
    def agg_mats_for_subplan(self, op_series, op_mats, subplan):
        if subplan['category'] == 'mastery':
            skill_idx = subplan['skillIndex']
            mastery_idx = subplan['masteryLevel'] - 1
            skill = op_series['skills'][skill_idx]
            mastery = skill['masteries'][mastery_idx]
            ingredients = {
                ingredient['id']: ingredient['quantity']
                for ingredient in mastery['ingredients']
            } 
        elif subplan['category'] == 'elite':
            idx = subplan['eliteLevel'] - 1
            ingredients = {
                ingredient['id']: ingredient['quantity']
                for ingredient in op_series['elite'][idx]['ingredients']
            } 
            
        elif subplan['category'] == 'skillLevel':
            ingredients = {}
        op_mats.update(ingredients)
            
    
    def prettify_mat_output(self, mats: Counter):
        return {
            self._item_by_id[item_id]: count
            for item_id, count in mats.items()
        }
    
    def penguin_stats_output(self, mats: Counter):
        # {,"items":[{"id":"30135","have":2,"need":4},{"id":"31033","have":5,"need":10}],}
        return {
            "@type": "@penguin-statistics/planner/config",
            "items": [{"id": key, "have": 0, "need": value} for key, value in mats.items()],
            "options": {"byProduct": False, "requireExp": False, "requireLmb": False},
            "excludes": []
        }
            
 

In [265]:
agg = OperatorMaterialAggregator()
mats = agg.aggregate_mats_by_rarity(rarities=[6])
agg.prettify_mat_output(mats)

{'Skill Summary - 1': 449,
 'Damaged Device': 56,
 'Ester': 78,
 'Skill Summary - 2': 1080,
 'Orirock Cube': 119,
 'Sugar': 95,
 'Polyketon': 88,
 'Oriron Cluster': 56,
 'Skill Summary - 3': 360,
 'Aketon': 75,
 'Loxic Kohl': 95,
 'Orirock': 72,
 'Polyester': 93,
 'Oriron': 100,
 'Integrated Device': 51,
 'Orirock Cluster': 77,
 'Coagulating Gel': 38,
 'Oriron Shard': 76,
 'Sugar Substitute': 72,
 'Device': 60,
 'Incandescent Alloy': 31,
 'Manganese Ore': 63,
 'Grindstone': 55,
 'Diketon': 68,
 'Sugar Pack': 21,
 'RMA70-12': 51,
 'Polyester Pack': 14,
 'Crystalline Component': 10}

In [266]:
# surtr_row = agg._op_df[agg._op_df['name'] == 'Surtr']
# #dir(surtr_row)
# surtr_row.index[0]
# surtr_row.iloc[0]['skills'][2]['masteries'][2]
# #surtr_row.iloc[]



In [267]:
operator_farm_plan
small_plan = {'Surtr': operator_farm_plan['Surtr']}

overall_mats, overall_mats_by_op = agg.agg_mats_for_farm_plan(operator_farm_plan)
agg.prettify_mat_output(overall_mats)

{'Skill Summary - 3': 553,
 'Orirock Concentration': 82,
 'Grindstone Pentahydrate': 49,
 'D32 Steel': 44,
 'Polyester Lump': 18,
 'White Horse Kohl': 56,
 'Polymerized Gel': 38,
 'Polymerization Preparation': 44,
 'LMD': 1960000,
 'Sniper Dualchip': 4,
 'Keton Colloid': 43,
 'Coagulating Gel': 8,
 'Bipolar Nanoflake': 40,
 'RMA70-24': 40,
 'Manganese Ore': 15,
 'Grindstone': 40,
 'Oriron Block': 21,
 'Sugar Lump': 12,
 'Polyester Pack': 6,
 'Manganese Trihydrate': 19,
 'Optimized Device': 27,
 'Loxic Kohl': 31,
 'Incandescent Alloy Block': 20,
 'Medic Dualchip': 7,
 'Guard Chip': 14,
 'Polyester': 26,
 'Device': 11,
 'Guard Dualchip': 11,
 'Oriron Cluster': 4,
 'Orirock Cube': 9,
 'Polyketon': 10,
 'Aketon': 23,
 'Orirock Cluster': 4,
 'Supporter Chip': 9,
 'Sugar': 5,
 'Supporter Dualchip': 7,
 'Incandescent Alloy': 17,
 'Specialist Dualchip': 4,
 'Crystalline Circuit': 4,
 'Vanguard Chip': 5,
 'Vanguard Dualchip': 4,
 'Caster Dualchip': 3,
 'Vanguard Chip Pack': 5,
 'Integrated Devi

In [270]:
penguin_output_raw = agg.penguin_stats_output(overall_mats)
print(json.dumps(penguin_output_raw))

{"@type": "@penguin-statistics/planner/config", "items": [{"id": "3303", "have": 0, "need": 553}, {"id": "30014", "have": 0, "need": 82}, {"id": "30094", "have": 0, "need": 49}, {"id": "30135", "have": 0, "need": 44}, {"id": "30034", "have": 0, "need": 18}, {"id": "30074", "have": 0, "need": 56}, {"id": "31014", "have": 0, "need": 38}, {"id": "30115", "have": 0, "need": 44}, {"id": "4001", "have": 0, "need": 1960000}, {"id": "3243", "have": 0, "need": 4}, {"id": "30054", "have": 0, "need": 43}, {"id": "31013", "have": 0, "need": 8}, {"id": "30125", "have": 0, "need": 40}, {"id": "30104", "have": 0, "need": 40}, {"id": "30083", "have": 0, "need": 15}, {"id": "30093", "have": 0, "need": 40}, {"id": "30044", "have": 0, "need": 21}, {"id": "30024", "have": 0, "need": 12}, {"id": "30033", "have": 0, "need": 6}, {"id": "30084", "have": 0, "need": 19}, {"id": "30064", "have": 0, "need": 27}, {"id": "30073", "have": 0, "need": 31}, {"id": "31024", "have": 0, "need": 20}, {"id": "3263", "have":

In [256]:
op_row = op_df[op_df["name"] == "Amiya"]

op_row["elite"][0]

# for idx in op_row.index:
#     op_skill_levels = op_row["skills"][idx]
#     for sk_lvl in op_skill_levels:
#         print(sk_lvl['masteries'])
    
    
    
# ingredients_by_level = op_row["skillLevels"][idx][0]["ingredients"]
#op_row["skillLevels"]
#op_row['skillLevels'][0]

#op_row.loc[['skillLevels']]

[{'eliteLevel': 1,
  'ingredients': [{'id': '4001',
    'name': 'LMD',
    'tier': 4,
    'quantity': 20000,
    'sortId': 10004},
   {'id': '3251',
    'name': 'Caster Chip',
    'tier': 3,
    'quantity': 3,
    'sortId': 400022},
   {'id': '30062',
    'name': 'Device',
    'tier': 2,
    'quantity': 4,
    'sortId': 100018},
   {'id': '30042',
    'name': 'Oriron',
    'tier': 2,
    'quantity': 4,
    'sortId': 100030}],
  'goalName': 'Elite 1',
  'goalCategory': 0},
 {'eliteLevel': 2,
  'ingredients': [{'id': '4001',
    'name': 'LMD',
    'tier': 4,
    'quantity': 120000,
    'sortId': 10004},
   {'id': '3253',
    'name': 'Caster Dualchip',
    'tier': 5,
    'quantity': 3,
    'sortId': 400006},
   {'id': '30014',
    'name': 'Orirock Concentration',
    'tier': 4,
    'quantity': 10,
    'sortId': 100012},
   {'id': '30073',
    'name': 'Loxic Kohl',
    'tier': 3,
    'quantity': 10,
    'sortId': 100005}],
  'goalName': 'Elite 2',
  'goalCategory': 0}]